# Solucion varios knapsack con otro metodo:

# Construccion de la matriz de funcion objetivo

Construiremos una matriz $Q_{objetivo}$ tal el problema $max\{  p^{t}x\}$ sea analogo a $min\{  x^{t}(-Q_{objetivo})x\}$

Esta matriz $Q$ se puede determinar de la siguiente manera:

 \begin{pmatrix}
    p_1 & 0 & \dots & 0 \\
    0 & p_2 & \dots & 0 \\
    \vdots & \vdots & \ddots & \vdots \\
    0 & 0 & \dots & p_n
  \end{pmatrix}

  Si hacemos las cuentas:

  $$  x^{t}(-Q_{objetivo})x = p_1x_1^2 + p_2x_2^2+ \dots + p_nx_n^2  $$
  y como $x_i \in [0, 1] \rightarrow x_i^2 = x_i$ 

  entonces:

  $$min\{ x^{t}(-Q_{objetivo})x\} = max\{ \sum_i p_ix_i\} = max\{p^{t}x\}$$

  > Observación: la dimension de la matriz es $n \times n$. Luego esto cambiará  

# Implementacion de la matriz de restriccion

Consideramos la restriccion: 

$w^{t}x \leq W$

Si $H_{B}$ es el Hamiltoniano de las restricciones:

$$H_{B} = A\left[ y_{L-1}(W + 1 - 2^{L-1}) + \sum_{i=0}^{L-2} 2^{i}y_{i}\ + \sum_{i=1}^{n} w_{i}x_{i} - W \right]^{2}$$

Lo que haremos es buscar una forma de representar $\frac{H_{B}}{A}$ como una forma cuadratica, y luego elegimos el valor de $A$ (representado por la variable $\alpha$ en el codigo).

## Necesidad de variables auxiliares
Para hallar esta matriz, debemos ampliar al vector $x$ ya que se necesitan agregar unas ciertas variables de slack $s_i$, donde $s_i = y_{i+1}$ en el Hamiltoniano. La cantidad de variables de slack a agregar es:

$$L =  \lfloor log_2(W)\rfloor+1$$

Entonces el vector $x$ queda:

$$x = [x_1 , x_2 , \dots, x_n, s_1, s_2,\dots,s_L ]$$

> Nuestra nueva matriz $Q_{objetivo}$ queda:

 \begin{pmatrix}
    Q_{objetivo} & 0 & \dots & 0 \\
    0 & 0_1 & \dots & 0 \\
    \vdots & \vdots & \ddots & \vdots \\
    0 & 0 & \dots & 0_L \\
  \end{pmatrix}

## Implementacion de la matriz de restriccion

Con esta aclaracion en mente, la matriz de restriccion de peso queda:

$$Q_{peso} = v^{t}v - 2 W D$$ 

Siendo $D$ la matriz diagonal con $v$ en su diagonal, y 

$$v = \left( w_1, w_2,\dots ,w_n, 2^0, 2^1,...,2^{L-2}, W+1 -2^{L-1} \right)$$

Notemos que:

$$\alpha x^{t}Q_{peso}x = x^{t}(\alpha Q_{peso})x = H_B$$

Como se buscaba

In [ ]:
import os
import time
import numpy as np
import pandas as pd
import math
from functions_dwave import *
import dimod
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Creamos la misma instancia de knapsack

profits = np.array([18, 15, 10, 10, 18])
weights = np.array([19, 13, 10, 17, 10])
data = dict({'weight': weights,'profit': profits})
num_items = len(profits)
max_weight = int(np.floor(num_items / 2 * np.mean(weights)))
df = pd.DataFrame(data)
print("-------------------------------------")
print("Choose items from: \n ")
print(df)
print(f"with a max weight of: {max_weight}\n ")
print("-------------------------------------")

In [ ]:
# Calculamos la cantidad de slacks
num_slack = int(math.trunc(np.log2(max_weight))) + 1
print("Cantidad de slacks de peso: ", num_slack)

In [ ]:
# Formulacion del problema
q_objetivo = get_Q_objetivo(profits, num_slack)
print("Matriz q_objetivo: \n", q_objetivo)

In [ ]:
q_peso = get_Q_Peso(max_weight, weights, num_slack)
print("Matriz q_peso: \n", q_peso)

In [ ]:
alpha = 80 #2/9
qubo = -q_objetivo + alpha * q_peso

print("Dimensiones del problema: ", qubo.shape)

In [ ]:
print("simmulating....")
sampleset = sendToDwave(qubo)
print("Filtering:")
feasibleSampleset = filterKnapSampleset(sampleset,  max_weight, weights,  num_slack)
bestSample = lowest_energy(feasibleSampleset)
solution = bestSample[0]
x_opt = solution[0:num_items] # le saco las slacks

In [ ]:
index_of_chosen = np.where(x_opt == 1)[0] # retorna los indices de los items elegidos.
output_df = df.iloc[index_of_chosen]
print("-------------------------------------")
print("Choosen items are: ")
print(output_df)
print("-------------------------------------")

# Variacion del problema:

vamos a resolver ahora una variacion, donde ademas del peso, tambien hay un volumen maximo, y cada item tiene un volumen asignado. La resolucion es totalmente analoga a la anterior

si $A$ es el volumen maximo, y los volumenes de cada item se representan en el vector:

$$[a_{1}, a_{2}, \dots, a_{n}]$$

Para integrarlo a nuestro modelo, con L variables de slack para la restriccion anterior: 

$$a = [a_{1}, a_{2}, \dots, a_{n}, 0_{1}, \dots, 0_{L}]$$

y la restriccion queda:

$$a^{t}x \leq A$$

A partir de aqui hacemos lo mismo que con la restriccion anterior, agregamos $M$ variables de slack, con:

$$M =  \lfloor log_2(A)\rfloor+1$$

La nueva matriz $Q_{objetivo}$ queda:

 \begin{pmatrix}
    Q_{objetivo} & 0 & \dots & 0 & 0 & \dots & 0\\
    0 & 0_1 & \dots & 0 & 0 & \dots & 0\\
    \vdots & \vdots & \ddots & \vdots & \vdots & \ddots & \vdots\\
    0 & 0 & \dots & 0_L & 0 & \dots & 0\\
    0 & 0 & \dots & 0 & 0_1 & \dots & 0 \\
    \vdots & \vdots & \ddots & \vdots & \vdots & \ddots & \vdots\\
    0 & 0 & \dots & 0 & 0 & \dots & 0_M \\
  \end{pmatrix}

y la matriz $Q_{peso}$ queda:

 \begin{pmatrix}
    Q_{peso} & 0 & \dots & 0 \\
    0 & 0_1 & \dots & 0 \\
    \vdots & \vdots & \ddots & \vdots \\
    0 & 0 & \dots & 0_M \\
  \end{pmatrix}

Asi, la matriz de restriccion de volumen queda: 

$$Q_{volumen} = v_{2}^{t}v_{2} - 2 A D_2$$

donde $D_2$ es una matriz diagonal con $v_2$ como diagonal principal y:

$$v_{2} = [a_{1}, a_{2}, \dots, a_{n}, 0_{1}, \dots, 0_{L}, 2^{0}, 2^{1}, \dots, 2^{M-2}, A + 1 - 2^{M-1}]$$

Finalmente, la matriz $Q$ a optimizar es:

$$Q = -Q_{objetivo} + \alpha Q_{peso} + \beta Q_{volumen}$$

Con $\alpha$ y $\beta$ parametros a elegir.



In [ ]:
# Creamos la misma instancia de knapsack

profits = np.array([18, 15, 10, 10, 18])
weights = np.array([19, 13, 10, 17, 10])
volumes = np.array([2, 1, 3, 4, 5]) #Elegimos 0 y 4
#volumes = np.array([12, 17, 14, 15, 11]) #Elegimos 0 y 4
# Si todos los volumenes son iguales, y la mejor solucion para el problema simplificado
# (knapsack original) cumple la restriccion de volumen, entonces esa misma deberia ser la solucion a este problema:
#volumes = np.array([20, 20, 20, 20, 20])
data = dict({'volumes': volumes, 'weight': weights,'profit': profits})
num_items = len(profits)
max_weight = int(np.floor(num_items / 2 * np.mean(weights)))
max_volume = int(np.floor(num_items / 2 * np.mean(volumes)))
# si max_volume >= 60, no cambia en la solucion de [1, 2, 4] para el problema simplificado
# un primer test que tiene que pasar esta solucion entonces es dar como resultado [1, 2, 4] con las variables actuales:
#max_volume = 70
df = pd.DataFrame(data)
print("-------------------------------------")
print("Choose items from: \n ")
print(df)
print(f"with a max weight of: {max_weight}\n ")
print(f"with a max volume of: {max_volume}\n ")
print("-------------------------------------")

In [ ]:
num_slack_weight = int(math.trunc(np.log2(max_weight))) + 1
print("Cantidad de slacks de peso: ", num_slack_weight)

num_slack_volume = int(math.trunc(np.log2(max_volume))) + 1
print("Cantidad de slacks de volumen: ", num_slack_volume)

num_slack = num_slack_volume + num_slack_weight

In [ ]:
q_objetivo = get_Q_objetivo(profits, num_slack)
print("Matriz q_objetivo: \n", q_objetivo)

q_peso = get_Q_Peso(max_weight, weights, num_slack_weight, num_slack)
print("Matriz q_peso: \n", q_peso)



In [ ]:
q_volumen = get_Q_Volumen(max_volume, volumes, num_slack_volume, num_slack)
print("Matriz 1_volumen: \n", q_volumen)

In [ ]:
alpha = 50 #2/9
beta = 50
qubo = -q_objetivo + alpha * q_peso + beta * q_volumen

print("Dimensiones del problema: ", qubo.shape)

In [ ]:
print("simmulating....")
sampleset = sendToDwave(qubo)
print("Filtering:")
feasibleSampleset = filterKnapSampleset(sampleset, max_weight, weights, max_volume, volumes, num_slack_weight ,num_slack)
bestSample = lowest_energy(feasibleSampleset)
solution = bestSample[0]
x_opt = solution[0:num_items] # le saco las slacks

In [ ]:
index_of_chosen = np.where(x_opt == 1)[0] # retorna los indices de los items elegidos.
output_df = df.iloc[index_of_chosen]
print("-------------------------------------")
print("Choosen items are: ")
print(output_df)
print("-------------------------------------")

Para resolver esta version del problema, con un $n$ items, un peso maximo de $W$ y un volumen maximo de $A$, se necesitan:

$N = n + L + M$ qbits

Donde

$$L = \lfloor log_2(W)\rfloor + 1$$
$$M = \lfloor log_2(A)\rfloor + 1$$

Son la cantidad de variables de slack agregadas para las restricciones de peso y volumen respectivamente.